We will work with the data provided by the [Open Data portal of the City of Rome](http://dati.comune.roma.it/). 
We use datasets related to registered population from other countries based on country of origin, sex and place of residence which is available for the years 2014, 2013 and 2012. In particular we will work with the following three sets available in CSV format (using semicolons as delimiter):
* [Popolazione straniera iscritta in anagrafe per municipio, sesso e cittadinanza. Al 31-12-2014](http://dati.comune.roma.it/cms/it/dettaglio_pop_e_societa.page?contentId=DTS542)
* [Popolazione straniera iscritta in anagrafe per municipio, sesso e cittadinanza. Al 31-12-2013](http://dati.comune.roma.it/cms/it/dettaglio_pop_e_societa.page?contentId=DTS725)
* [Popolazione straniera iscritta in anagrafe per municipio, sesso e cittadinanza. Al 31-12-2012](http://dati.comune.roma.it/cms/it/dettaglio_pop_e_societa.page?contentId=DTS728)

All three datasets share the same format:
- Country of Origin
- Continent
- For each Municipio of Rome (the city is devided in 15 administrative regions) a breakdown

 * Males (M)
 * Females (M)
 * Total (TOT or MF)